In [8]:
from elasticsearch import Elasticsearch

In [9]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create('es0', body={
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "fields": {
                    "raw": {
                        "type": "keyword"
                    }
                }
            }
        }
    }
})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [10]:
es.index('es0', id=0, body={"content": "Apple Mac"})
es.index('es0', id=1, body={"content": "Apple iPad"})
es.index('es0', id=2, body={"content": "Apple employee like Apple Pie and Apple juice"})
es.index('es0', id=3, body={"content": "Apple employee like Apple Pie and Apple juice and Apple snack"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '3',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 3,
 '_primary_term': 1}

In [11]:
es.search(index='es0', body={
    "query": {
        "bool": {
            "must": [
                {
                    "boosting": {
                        "positive": {
                            "match": {
                                "content": {
                                    "query": "apple"
                                }
                            }
                        },
                        "negative": {
                            "match": {
                                "content": {
                                    "query": "pie"
                                }
                            }
                        },
                        "negative_boost": 0.5
                    }
                }
            ],
            "must_not": [
                {
                    "match": {
                        "content": {
                            "query": "snack"
                        }
                    }
                }
            ]
        }
    }
})

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [13]:
es.search(index='es0', body={
    "query": {
        "bool": {
            "must": [
                {
                    "function_score": {
                        "query": {
                            "match": {
                                "content": {
                                    "query": "apple"
                                }
                            }
                        },
                        "script_score": {
                            "script": {
                                "params": {
                                    "a": 100,
                                    "b": 0.1
                                },
                                "source": """
                                boolean contains_pie = doc['content.raw'].value.contains("Pie");
                                if (contains_pie) {
                                  return _score / params.a
                                } else {
                                  return _score / params.b
                                }
                                """
                            }
                        },
                        "boost_mode": "replace"
                    }
                }
            ],
            "must_not": [
                {
                    "match": {
                        "content": {
                            "query": "snack"
                        }
                    }
                }
            ]
        }
    }
})

RequestError: RequestError(400, 'search_phase_execution_exception', 'runtime error')

In [7]:
es.indices.delete('es0')

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [es0]', es0, index_or_alias)